# Lecture 4.4 - Basics of cleaning messy text files 
## Part 1 - Getting the know the file and removing unwanted lines

In this lecture, we will go over a number of cases of messy data, and how to use Python to fix these problems.  This includes

1. Use VSCode to inspect the file and identify problems.
1. Removing unwanted lines.

## Example - 9/11 Deaths 

I have included a file named `911_Deaths.csv`.  Download this file from D2L and open it in VScode.  Be sure to turn on **View > Render Whitespace**, which I find useful when working with messy data.

> Not actually comma-separated
  Inclusion of commas in names, etc. may be problematic with interpreting
  Values are on separate lines rather than one
  Periodic page break - 1 of 91, etc.

## Reading in the lines of the file

I have included a file named `911_Deaths.csv`.  Open this file and split the body into a list called `lines`.

In [1]:
with open("911_Deaths.csv", "r") as fi:
    lines = fi.readlines()
lines[:5]

['Gordon M. Aamoth, Jr.\n',
 ' 32\n',
 " Sandler O'Neill + Partners, World Trade Center.\n",
 'Edelmiro Abad\n',
 ' 54\n']

In [ ]:
#import pandas as pd
#f = pd.read_csv("911_Deaths.csv") # long ParserError - yeah, this is a bad file

## Step 1: Remove any extraneous lines

In [2]:
example_w_extra = lines[96:107]
example_w_extra

['Terrance Andre Aiken\n',
 ' 30\n',
 ' Marsh&McLennan consultant, World Trade Center.\n',
 '\n',
 '\n',
 '\t\t\t\t\t\t\t\t\t\t1 of 91\n',
 '\n',
 '\n',
 'Godwin O. Ajala\n',
 ' 33\n',
 ' Summit Security Services, Inc., World Trade Center, died 9/15/01.\n']

## Methods for removing extra lines.

* `strip` and check for a length of 0
* Check if some substring is `in` the line.
* Create a regular expression to match the line type

In [5]:
[l for l in lines if l.startswith("\t")][-5:]

['\t\t\t\t\t\t\t\t\t\t86 of 91\n',
 '\t\t\t\t\t\t\t\t\t\t87 of 91\n',
 '\t\t\t\t\t\t\t\t\t\t88 of 91\n',
 '\t\t\t\t\t\t\t\t\t\t89 of 91\n',
 '\t\t\t\t\t\t\t\t\t\t90 of 91\n']

<h2> <font color="red"> Exercise 4.4.2 </font> </h2>

Rewrite a predicate function (i.e. Boolean lambda) called `is_blank` that returns `True` for blank lines and `False` otherwise.

In [6]:
example_blank_line = lines[99]
example_blank_line == "\n" # True
example_blank_line.startswith("\n") # True

True

In [7]:
is_blank = lambda line: line.startswith("\n")

<h2> <font color="red"> Exercise 4.4.3 </font> </h2>

Rewrite a predicate function called `is_page_num_line` that returns `True` for the page number lines and `False` otherwise.

In [8]:
example_page_num_line = lines[101]
example_page_num_line

'\t\t\t\t\t\t\t\t\t\t1 of 91\n'

In [9]:
is_page_num_line = lambda line: line.startswith("\t")
is_page_num_line(example_page_num_line) # True
is_page_num_line(example_blank_line)

False

<h2> <font color="red"> Exercise 4.4.4 </font> </h2>

Identify some situation were your approach might fail.

>is_blank should work, as having anything after the \n would necessarily put it on the next line

>is_page_num_line would be a problem if there were any meaningful lines that started with a \t (but there aren't any currently)

## Be as specific as possible

When removing lines, be very specific so you don't lose data!

<h2> <font color="red"> Exercise 4.4.5 </font> </h2>

Complete the following steps.

#### Describe the pattern in words

In [10]:
example_page_num_line = lines[101]
example_page_num_line

'\t\t\t\t\t\t\t\t\t\t1 of 91\n'

> starts with a bunch of \t, then (num) of 91

#### Write a regular expression to `match` the problem

In [31]:
import re
pattern = re.compile(r"\t")
pattern.match(example_page_num_line)

<re.Match object; span=(0, 1), match='\t'>

In [34]:
[l for l in lines if pattern.match(l)][:5]

['\t\t\t\t\t\t\t\t\t\t1 of 91\n',
 '\t\t\t\t\t\t\t\t\t\t2 of 91\n',
 '\t\t\t\t\t\t\t\t\t\t3 of 91\n',
 '\t\t\t\t\t\t\t\t\t\t4 of 91\n',
 '\t\t\t\t\t\t\t\t\t\t5 of 91\n']

#### Why is `match` better than `search` for this task?

> We know the lines we're looking for will start with the \t, so we can use match and skip the starts-with character

## Combining the two predicate functions

In [12]:
needs_removal = lambda l: is_blank(l) or is_page_num_line(l)

In [13]:
[(l, needs_removal(l)) for l in example_w_extra ]

[('Terrance Andre Aiken\n', False),
 (' 30\n', False),
 (' Marsh&McLennan consultant, World Trade Center.\n', False),
 ('\n', True),
 ('\n', True),
 ('\t\t\t\t\t\t\t\t\t\t1 of 91\n', True),
 ('\n', True),
 ('\n', True),
 ('Godwin O. Ajala\n', False),
 (' 33\n', False),
 (' Summit Security Services, Inc., World Trade Center, died 9/15/01.\n',
  False)]

#### Removing the extraneous lines

In [14]:
culled_lines = [l for l in lines if not needs_removal(l)]
culled_lines[:100]

['Gordon M. Aamoth, Jr.\n',
 ' 32\n',
 " Sandler O'Neill + Partners, World Trade Center.\n",
 'Edelmiro Abad\n',
 ' 54\n',
 ' Brooklyn, N.Y., Fiduciary Trust Company International, World Trade Center.\n',
 'Marie Rose Abad\n',
 ' 49\n',
 ' Keefe, Bruyette&Woods, Inc., World Trade Center.\n',
 'Andrew Anthony Abate\n',
 ' 37\n',
 ' Melville, N.Y., Cantor Fitzgerald, World Trade Center.\n',
 'Vincent Paul Abate\n',
 ' 40\n',
 ' Brooklyn, N.Y., Cantor Fitzgerald, World Trade Center.\n',
 'Laurence Christopher Abel\n',
 ' 37\n',
 ' New York City, Cantor Fitzgerald, World Trade Center.\n',
 'Alona Abraham\n',
 ' 30\n',
 ' Ashdod, Israel, Passenger, United 175, World Trade Center.\n',
 'William F. Abrahamson\n',
 ' 55\n',
 ' Westchester County, N.Y., Marsh&McLennan Companies, Inc., World Trade Center.\n',
 'Richard Anthony Aceto\n',
 ' 42\n',
 ' Marsh&McLennan Companies, Inc., World Trade Center.\n',
 'Heinrich Bernhard Ackermann\n',
 ' 38\n',
 ' Aon Corporation, World Trade Center.\n',
 'Pa

In [20]:
keep_line = lambda l: not needs_removal(l)
culled_lines = [l for l in lines if keep_line(l)]
culled_lines[:10]

['Gordon M. Aamoth, Jr.\n',
 ' 32\n',
 " Sandler O'Neill + Partners, World Trade Center.\n",
 'Edelmiro Abad\n',
 ' 54\n',
 ' Brooklyn, N.Y., Fiduciary Trust Company International, World Trade Center.\n',
 'Marie Rose Abad\n',
 ' 49\n',
 ' Keefe, Bruyette&Woods, Inc., World Trade Center.\n',
 'Andrew Anthony Abate\n']

## Which reads better?

In [21]:
culled_lines = [l for l in lines if not needs_removal(l)]
culled_lines = [l for l in lines if keep_line(l)]

#### Make it a function!

In [24]:
culled_lines = [l for l in lines if keep_line(l)]
culled_lines[:10]

['Gordon M. Aamoth, Jr.\n',
 ' 32\n',
 " Sandler O'Neill + Partners, World Trade Center.\n",
 'Edelmiro Abad\n',
 ' 54\n',
 ' Brooklyn, N.Y., Fiduciary Trust Company International, World Trade Center.\n',
 'Marie Rose Abad\n',
 ' 49\n',
 ' Keefe, Bruyette&Woods, Inc., World Trade Center.\n',
 'Andrew Anthony Abate\n']

In [17]:
# Key
cull_lines = pipeable(lambda L: [l for l in L if keep_line(l)])
culled_lines = cull_lines(lines)
culled_lines[:5]

NameError: name 'pipeable' is not defined

# Clean Code Rule

### *When writing Boolean expressions, avoid negatives whenever possible.*

<h2> <font color="red"> Exercise 4.4.5 -- Saving our work </font> </h2>

Now we will save our work by writing our progress to a text file.

Complete the following steps.

#### Step 1 -- `'\n'` join the `culled_lines`

In [25]:
out_text = "\n".join(culled_lines)

#### Step 2 -- Write the string to a file called `911_deaths_extra_removed.csv`

In [26]:
with open("911_deaths_extra_removed.csv", "w+") as of:
    of.write(out_text)

In [27]:
! cat 911_deaths_extra_removed.csv | head -20

Gordon M. Aamoth, Jr.

 32

 Sandler O'Neill + Partners, World Trade Center.

Edelmiro Abad

 54

 Brooklyn, N.Y., Fiduciary Trust Company International, World Trade Center.

Marie Rose Abad

 49

 Keefe, Bruyette&Woods, Inc., World Trade Center.

Andrew Anthony Abate

cat: write error: Broken pipe
